# **Рынок заведений общественного питания Москвы**

Инвесторы из фонда «Shut Up and Take My Money» решили попробовать себя в новой области и открыть заведение общественного питания в Москве. Заказчики ещё не знают, что это будет за место: кафе, ресторан, пиццерия, паб или бар, — и какими будут расположение, меню и цены.
Для начала они просят вас — аналитика — подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места.

**Цель исследования:** найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места

**Ход исследования:**

- Изучение общей информации датасета
- Предобработка данных
- Анализ данных
- Детализация данных открытие кофейни


# **Описание данных**

**Файл `moscow_places.csv`:**

`name` — название заведения;

`address` — адрес заведения;

`category` — категория заведения, например «кафе», «пиццерия» или «кофейня»;

`hours` — информация о днях и часах работы;

`lat` — широта географической точки, в которой находится заведение;

`lng` — долгота географической точки, в которой находится заведение;

`rating` — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);

`price` — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее;

`avg_bill` — строка, которая хранит среднюю стоимость заказа в виде диапазона, например:
  
«Средний счёт: 1000–1500 ₽»;
«Цена чашки капучино: 130–220 ₽»;
«Цена бокала пива: 400–600 ₽».
и так далее;

`middle_avg_bill` — число с оценкой среднего чека, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Средний счёт»:
  
Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.
Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.
Если значения нет или оно не начинается с подстроки «Средний счёт», то в столбец ничего не войдёт.

`middle_coffee_cup` — число с оценкой одной чашки капучино, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Цена одной чашки капучино»:
  
Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.
Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.
Если значения нет или оно не начинается с подстроки «Цена одной чашки капучино», то в столбец ничего не войдёт.

`chain` — число, выраженное 0 или 1, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки):
0 — заведение не является сетевым
1 — заведение является сетевым

`district` — административный район, в котором находится заведение, например Центральный административный округ;

`seats` — количество посадочных мест.


## **Откроем файл и изучим общую информацию**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objects as go
import plotly.express as px
import warnings
import folium
import json
from folium import Map, Choropleth, Marker
from folium.plugins import MarkerCluster

In [ ]:
try:
    moscow_exp = pd.read_csv(r'C:\Users\niksmns\Desktop\moscow_places\moscow_places.csv')
    state_map = r'https://code.s3.yandex.net/data-analyst/admin_level_geomap.geojson'
except:
    moscow_exp = pd.read_csv('/datasets/moscow_places.csv')
    state_map = '/datasets/admin_level_geomap.geojson'

In [ ]:
def general_info(data):
    display(data.head(10))
    display(data.info())
    display(pd.DataFrame(round(data.isna().mean()*100,)).style.background_gradient('coolwarm'))
    display('Количество дубликатов:', data.duplicated().sum())
    display(data.columns)
    display(data.describe())

In [ ]:
general_info(moscow_exp)

In [ ]:
work = moscow_exp['lat']astype + moscow_exp['lng']
work

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">
    
Изучили информацию о датасете, было найдено много пропусков.
Также в ходе ручной проверки на достоверность данных было выявлено много некоректоной информации. Скорее всего этому послужила ошибка ввода данных и неверно настроенные алгоритмы программы
</div>

## **Предобработка данных**

In [ ]:
# создадим колонку с улицами разделив строки методом split
moscow_exp['street'] = moscow_exp['address'].str.split(',').str[1]
moscow_exp['street']

In [ ]:
# посмотрим какие есть значения
moscow_exp['hours'].value_counts()

In [ ]:
# функция для категорий графика работы 24/7
def hours_category(row):
    try:
        if 'ежедневно' and 'круглосуточно' in row:
            return True
        else:
            return False
    except:
        return 'неизвестно'

In [ ]:
# применяем нашу функцию 
moscow_exp['is_24/7'] = moscow_exp['hours'].apply(hours_category).astype('bool')
# проверяем
#moscow_exp[['hours', 'is_24/7']]

In [ ]:
# проверяем корректность функции
#moscow_exp[['hours', 'is_24/7']].loc[moscow_exp['is_24/7'] == 'неизвестно'].head(5)

In [ ]:
# ставим заглушки в пропусках 
moscow_exp['hours'] = moscow_exp['hours'].fillna('неизвестно')
moscow_exp['price'] = moscow_exp['price'].fillna('неизвестно')
moscow_exp['avg_bill'] = moscow_exp['avg_bill'].fillna('неизвестно')
moscow_exp['seats'] = moscow_exp['seats'].astype(int, errors='ignore')
moscow_exp['middle_avg_bill'] = moscow_exp['middle_avg_bill'].astype(int, errors='ignore')
moscow_exp['middle_coffee_cup'] = moscow_exp['middle_coffee_cup'].astype(int, errors='ignore')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 8))

sns.boxplot(data=moscow_exp, x='category', y='seats', ax=axs[0])
axs[0].tick_params(axis='x', rotation=25)
axs[0].tick_params(axis='y', rotation=25)
axs[0].set_title('График ящик с усами для количества мест по категориям заведения V1')
axs[0].set_xlabel('Категория заведения')
axs[0].set_ylabel('Кол-во мест')
sns.boxplot(data=moscow_exp, x='category', y='seats', ax=axs[1]).set(ylim=(0, 400))
axs[1].tick_params(axis='x', rotation=25)
axs[1].tick_params(axis='y', rotation=25)
axs[1].set_title('График ящик с усами для количества мест по категориям заведения V2')
axs[1].set_xlabel('Категория заведения')
axs[1].set_ylabel('Кол-во мест')
plt.show()

In [ ]:
# создаем столбец для поиска неявных дубликатов
moscow_exp['address'] = moscow_exp['address'].str.lower()
moscow_exp['name'] = moscow_exp['name'].str.lower()
moscow_exp['duplicate'] = moscow_exp['address'] + moscow_exp['name']

In [ ]:
# смотрим неявные дубликаты
moscow_exp[moscow_exp['duplicate'].duplicated() == True]['duplicate']

In [ ]:
moscow_exp['duplicate'] = moscow_exp[moscow_exp['duplicate'].duplicated() == False]['duplicate']

In [ ]:
moscow_exp = moscow_exp.drop(labels=[215, 1511, 2420, 3109], axis=0)
moscow_exp = moscow_exp.drop(columns='duplicate')

In [ ]:
# выявляем неявные дубликаты
moscow_exp[moscow_exp['name'].str.contains('домино')]['name'].unique()
# исправляем 
moscow_exp['name'] = moscow_exp['name'].replace(['доминос пицца', "домино'с" ], "домино'с пицца")
# проверяем
moscow_exp[moscow_exp['name'].str.contains('домино')]['name'].unique()

In [ ]:
# выявляем неявные дубликаты
moscow_exp[moscow_exp['name'].str.contains('яндекс')]['name'].unique()
# исправляем 
moscow_exp['name'] = moscow_exp['name'].replace('яндекс.лавка', 'яндекс лавка')
# проверяем
moscow_exp[moscow_exp['name'].str.contains('яндекс')]['name'].unique()

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Поменяли тип данных у колонок где смогли. Заполнили пропуски заглушками. Также добавили столбцы улиц заведений и столбец работы 24/7 и удалили неявные дубликаты
</div>

## **Анализ данных**

### *Количество заведений по категориям*

In [ ]:
mos_category_cnt = (
    moscow_exp.pivot_table(index='category', values='name', aggfunc='count')
    .reset_index())
mos_category_cnt.sort_values(by='name', ascending=False)
mos_category_cnt['percent'] = round(mos_category_cnt['name'] / mos_category_cnt['name'].sum() * 100, 2)
mos_category_cnt = mos_category_cnt.sort_values(by='percent', ascending=False)

In [ ]:
fig = px.bar(mos_category_cnt, x=mos_category_cnt['category'], y=mos_category_cnt['name'] ,text='percent')

fig.update_traces(textposition='inside', textangle=0)
fig.update_layout(title='Количество заведений по категориям и их доли',
                  xaxis_title='Категория заведений',
                  yaxis_title='Кол-во заведений')

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">
Видим, что больше всего заведений в Москве - это кафе, рестораны и кофейни.
Кафе и рестораны  занимают ~1/4 заведений Москвы, также видим, что у кофеен большая доля заведений, вероятно это связано с "четвертой волной", когда стало переосмысление кофе как продукта, и оно превратилось в науку, спорт и искусство.
Люди стали чаще открывать кофейни и кофейные точки, где-то осталась философия этой четвертой волны, а где-то осталась только коммерция. Также открытие простенькой кофейни не столько финансово затратно, как открытие других категорий заведений.
</div>

### *Посадочные места заведений по категориям*

In [ ]:
mos_seats = moscow_exp.pivot_table(index='category', values='seats', aggfunc='median').reset_index().sort_values(by='seats', ascending=False)
fig = px.bar(mos_seats, x=mos_seats['category'], y=mos_seats['seats'], text_auto='.0f')

fig.update_traces(textposition='inside', textangle=0)
fig.update_layout(title='Среднее количество мест по категориям',
                  xaxis_title='Категория заведений',
                  yaxis_title='Среднее кол-во мест')

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Поскольку в данных явно какие-то неккореткные аномальные сведения о количестве посадочных мест, выявлили среднее количество посадочных мест через медиану.
Выяснилось, что больше всего мест имеют рестораны, бары/пабы и кофейни.
</div>

### *Сетевые и несетевые заведения*

In [ ]:
mos_chain = moscow_exp.pivot_table(index='chain', values='name', aggfunc='count').reset_index()
fig = go.Figure(data=[go.Pie(labels=mos_chain['chain'].map({1: 'сетевые заведения', 0: 'несетевые заведения'}),
                             values=mos_chain['name'])])
fig.update_layout(title_text='Доли сетевых/несетевых заведений')
fig.show()

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Несетевых заведений в Москве больше, процентная доля их составляет - 61.9%, сетевых же заведений поменьше, их доля составляет - 38.1%
</div>

In [ ]:
mos_chain_categ = moscow_exp.pivot_table(index='category', values='name', columns='chain', aggfunc='count').reset_index()
mos_chain_categ['total_cnt'] = mos_chain_categ[0] + mos_chain_categ[1]
mos_chain_categ['percent_chain'] = round(mos_chain_categ[1] / mos_chain_categ['total_cnt'] * 100, 2)
mos_chain_categ = mos_chain_categ.sort_values(by='percent_chain', ascending=False)
mos_chain_categ

In [ ]:
fig = px.bar(mos_chain_categ, x=mos_chain_categ['category'], y=mos_chain_categ['percent_chain'], text_auto='.2f')

fig.update_traces(textposition='inside', textangle=0)
fig.update_layout(title='Доли сетевых заведений в рамках одной категории',
                  xaxis_title='Категория заведения',
                  yaxis_title='Процентные пункты')

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Булочные, пиццерии и кофейни занимают большую часть сетевых заведений по отношению не к сетевым
</div>

### *Топ-15 сетевых заведений*

In [ ]:
mos_chain_top_15 = (
    moscow_exp.query('chain == 1')
    .groupby(['category','name']).agg({'name':'count'}))
mos_chain_top_15.columns = ['count']
mos_chain_top_15 = mos_chain_top_15.sort_values(by='count', ascending=False).head(15).reset_index()
mos_chain_top_15

In [ ]:
fig = px.bar(mos_chain_top_15, x='name', y='count', text_auto='.2f', color='category')

fig.update_traces(textposition='inside', textangle=0)
fig.update_layout(title='Доли сетевых заведений в рамках одной категории',
                  xaxis_title='Категория заведения',
                  yaxis_title='Процентные пункты')

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Как видим, здесь очень много знакомых нам сетевых заведений, среди них очень много кофеен. Также большую часть занимает сетевые пицерии и рестораны
</div>

### *Административные районы*

In [ ]:
mos_district_categ = moscow_exp.pivot_table(index='district', columns='category', values='name', aggfunc='count')
mos_district_categ['total'] = mos_district_categ[['бар,паб','булочная','быстрое питание','кафе', 'кофейня', 'пиццерия', 'ресторан', 'столовая']].sum(axis=1)
mos_district_categ = mos_district_categ.sort_values(by='total', ascending=False)
mos_district_categ

In [ ]:
mos_district_categ = mos_district_categ.drop(columns='total', axis=1)

In [ ]:
fig = px.bar(mos_district_categ, text_auto='.0f', orientation='h')
fig.update_traces(textposition='inside', textangle=0)
fig.update_layout(title='Количество заведений по округам и категориям заведений',
                  xaxis_title='Количество заведений',
                  yaxis_title='Округа Москвы')

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Можем заметить, что в центре больше всего количества заведений каждого типа, в других же округах плюс-минус распределение  одинаковое за исключением Северо-Западного административного округа, там меньше всего заведений почти всех категорий.
</div>

### *Распределение средних рейтингов по категориям заведений*

In [ ]:
moscow_exp.pivot_table(index='category', values='rating', aggfunc=['count', 'mean'])

In [ ]:
with warnings.catch_warnings():# воспользуемся библиотекой warnings, чтобы убрать сообщения об ошибках
    warnings.simplefilter("ignore", category=UserWarning)
    plt.figure(figsize=(16,8)) 
    ax = sns.swarmplot(y='rating', x='category', alpha=0.5, data=moscow_exp) # воспользуемся категориальной точечной диаграммой
    ax.set_title('Распределение средних рейтингов по категориям заведений')
    ax.set_xlabel('Категория заведения')
    ax.set_ylabel('Рейтинг')
    ax.tick_params(axis='x', rotation=45)
    ax.tick_params(axis='y', rotation=45)

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Видим, что основное количество оценок приходится на кафе, рестораны и кофейни. Разнообразие же в рейтинге обладают кафе, рестораны и заведения быстрого питания. Усредненные же рейтиги различаются не очень сильно по всем категориям, но булочные, столовые и заведения быстрого питания меньше всего получают самые высокие оценки.
</div>

### *Фоновая картограмма*

In [ ]:
# координаты центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423
# создаем карту Москвы
mos_map = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_map,
    data=moscow_exp,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(mos_map)

marker_cluster = MarkerCluster().add_to(mos_map)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
moscow_exp.apply(create_clusters, axis=1)

# выводим нашу карту
mos_map


<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Можем выделить, что почти все регионы имеет среднюю оценку в 4+. Выделяются у нас тут два региона: Северо-Восточный административный округ, который имеет больше всего положительных оценок заведений, и Восточный административный округ, который имеет меньше положительных оценок и средний рейтинг можно назвать удовлетворительным.
</div>

### *Топ-15 улиц по количеству заведений*

In [ ]:
mos_street_top15 = (
    moscow_exp.pivot_table(index='street', columns='category', values='name', aggfunc='count')
)
mos_street_top15['total'] = mos_street_top15[['бар,паб','булочная','быстрое питание','кафе', 'кофейня', 'пиццерия', 'ресторан', 'столовая']].sum(axis=1)
mos_street_top15 = mos_street_top15.sort_values(by='total', ascending=False).head(15)
mos_street_top15

In [ ]:
mos_street_top15 = mos_street_top15.drop(columns=['total'], axis=1)
fig = px.bar(mos_street_top15, text_auto='.0f', orientation='h')
fig.update_traces(textposition='inside', textangle=0)
fig.update_layout(title='Количество заведений по улицами и категориям',
                  xaxis_title='Округа Москвы',
                  yaxis_title='Сумма средних чеков')

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">
Больше всего находится заведений на проспекте Мира, меньше же всего находится на Пятницкой улицы.
Думаю такое количество связано с местоположением и протяженностью улиц
</div>

### *Улицы с одним объектом общепита*

In [ ]:
msones = moscow_exp
msones['name_cnt'] = moscow_exp['name']
mos_one_street = msones.pivot_table(index='street', values='name_cnt', aggfunc='count')
mos_one_street = mos_one_street.reset_index().query('name_cnt == 1')
mac = moscow_exp[['name','lat', 'lng', 'street', 'rating', 'category', 'district']]
merge_one_st = mos_one_street.merge(mac)
merge_one_st

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
mos_map = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(mos_map)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster

Choropleth(
    geo_data=state_map,
    data=merge_one_st,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='RdPu',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(mos_map)

marker_cluster = MarkerCluster().add_to(mos_map)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['category']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
merge_one_st.apply(create_clusters, axis=1)

# выводим карту
mos_map

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Как заметно, в центре больше всего скоплений одного заведения на одну улицу, скорее всего это связано с большим количеством коротких проездов и переулков, да еще и играет роль, что в центре естественно будет больше заведений. Далее по мере удаления от ЦФО, наблюдаем тенденцию уменьшения кол-ва заведения и улицы становятся длиннее из-за этого вероятность одного заведения на одной улице значительно уменьшается, да и заведений становится меньше. Также заметно, что в парковых, набережных и железнодорожных  зонах чаще бывает одно заведение.
</div>

### *Значения средних чеков по округам*

In [ ]:
median_district = moscow_exp.pivot_table(index='district', values='middle_avg_bill', aggfunc='median').reset_index()
median_district.columns = ['district', 'middle_avg_district']
merge_median_dis = median_district.merge(moscow_exp)
merge_median_dis

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
mos_dis = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(mos_dis)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster

Choropleth(
    geo_data=state_map,
    data=merge_median_dis,
    columns=['district', 'middle_avg_district'],
    key_on='feature.name',
    fill_color='RdPu',
    fill_opacity=0.8,
    legend_name='Медианный средний чек по районам',
).add_to(mos_dis)

marker_cluster = MarkerCluster().add_to(mos_dis)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['category']} {row['rating']} {row['avg_bill']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
merge_median_dis.apply(create_clusters, axis=1)

# выводим карту
mos_dis

In [ ]:
median_dis_categ = merge_median_dis.pivot_table(index='district', columns='category', values='middle_avg_bill', aggfunc='median')
median_dis_categ

In [ ]:
fig = px.bar(median_dis_categ, text_auto='.0f', orientation='h')
fig.update_traces(textposition='inside', textangle=0)
fig.update_layout(title='Средний чек по округам и категориям заведений',
                  xaxis_title='Сумма средних чеков',
                  yaxis_title='Округа Москвы')

In [ ]:
display(moscow_exp['seats'].corr(moscow_exp['middle_avg_bill']))
display(moscow_exp['rating'].corr(moscow_exp['middle_avg_bill']))
display(moscow_exp['lng'].corr(moscow_exp['middle_avg_bill']))
display(moscow_exp['lat'].corr(moscow_exp['middle_avg_bill']))

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Проверка пр корреляции особо ничего нам не дала, а значит средний чек зависит от особенностей районов Москвы.
Видим, что ЦАО и ЗАО имеют наибольший средний чек, скорее всего это связано с тем, что это самые престижные округа в Москве, на них расположены много важных объектов и достопримечательностей. Из-за этого там живут чаще состоятельные люди, да и много людей приезжает в такие места. Самыми же худшими округами являются СВАО, ЮАО и ЮВАО в этих регионах. Они являются одними из самых густонаселенных округов где построено много новостроек, и имеет проблемы с экологией из-за предприятий. Поэтому в этих районах меньше дорогих ресторанов, да и покупательская способность другая
</div>

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

<b>Итак, выяснили, что в Москве:</b>

- больше всего заведений: кафе, рестораны, кофейни
- больше всего мест в заведениях: рестораны, бары/пабы и столовые
- больше всего несетевых заведений(61.9%), сетевых меньше(38.1%)
- больше всего по количеству сетевых заведений являются кофейни, далее пиццерии, а за ними и рестораны
- самая большая сетка из Топ-15 сетевых - Шоколадница, самая маленькая Кофемания
- самый большой округ по заведениям Центральный административный округ, а самый маленький Северо-Западный административный округ
- усредненные рейтиги различаются не очень сильно по всем категориям. Булочные, столовые и заведения быстрого питания меньше всего получают самые высокие оценки
- северо-Восточный административный округ имеет больше всего положительных оценок заведений. Восточный административный округ, имеет меньше положительных оценок
- местоположение и протяженность улиц влияет на количество заведений на них
- на коротких улицах, проездах и переулках чаще можно встретить только одно заведение, особенно в центре
- на средний чек в округах влияет особенности этих округов

</div>

## **Открытие кофейни**

### *Количество кофеен и их рейтинг по округам*

In [ ]:
moscow_coffee = moscow_exp.query('category == "кофейня"')
print('В Москве', moscow_coffee['name'].count(), 'кофеен')

In [ ]:
mc_pivot_cnt = (
    moscow_coffee.pivot_table(index='district', values='name', aggfunc='count')
    .reset_index())

mc_pivot_rating = (
    moscow_coffee.pivot_table(index='district', values='rating', aggfunc=['mean','median'])
    .reset_index()
    .droplevel(1, axis=1))
mc_pivot = mc_pivot_cnt.merge(mc_pivot_rating)
mc_pivot.columns = ['Округ', 'Количество кофеен', 'Средний рейтинг', 'Медианный рейтинг']
mc_pivot.sort_values(by='Средний рейтинг', ascending=False)

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
mos_cofeee = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту

marker_cluster = MarkerCluster().add_to(mos_cofeee)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']}  {row['rating']} {row['middle_coffee_cup']} {row['is_24/7']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
moscow_coffee.apply(create_clusters, axis=1)

heatmap = moscow_coffee[['lat', 'lng']]

folium.plugins.HeatMap(heatmap).add_to(mos_cofeee)
# выводим карту
mos_cofeee

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Как видим, в Москве 1413 кофееен. Больше всего кофеен сосредоточено в Центральном, Северном и Северо-Восточном административном округе, меньше всего кофеен находится в Северо-Западном, Юго-Восточном, Юго-Западном административном округе.
Из особенностей размещение видим, что много кофеен открывают у жилищного комплекса, ТЦ и ТРЦ, учебных заведений, бизнес-центрах и арт-пространств, то есть где всегда есть поток людей.
Средний рейтинг округа больше всего в ЦАО, а меньше всего в ЗАО.
</div>

### *Круглосуточные кофейни*

In [ ]:
mos_cof_24_7 = moscow_exp[(moscow_exp['is_24/7'] == True) & (moscow_exp['category'] == 'кофейня')]
print('Кофеен 24/7:', mos_cof_24_7['name'].count())

In [ ]:
mos_chain = mos_cof_24_7.pivot_table(index='district', columns='chain', values='name', aggfunc='count')
mos_chain['total'] = mos_chain[[0, 1]].sum(axis=1)
mos_chain.sort_values(by='total', ascending=False)

In [ ]:
mos_chain = mos_chain.drop(columns='total', axis=1)

In [ ]:
fig = px.bar(mos_chain, text_auto='.0f', orientation='h')
fig.update_traces(textposition='inside', textangle=0)
fig.update_layout(title='Средний чек по округам и категориям заведений',
                  xaxis_title='Количество кофеен',
                  yaxis_title='Округа Москвы')

In [ ]:
mos_chain_pie = mos_cof_24_7.pivot_table(index='chain', values='name', aggfunc='count').reset_index()
fig = go.Figure(data=[go.Pie(labels=mos_chain_pie['chain'].map({1: 'сетевые заведения', 0: 'несетевые заведения'}),
                             values=mos_chain_pie['name'])])
fig.update_layout(title_text='Доли сетевых/несетевых заведений')
fig.show() 

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

91 - это число кофеен, которые работают 24/7. Из них видим, что сетевые кофейни занимают значительно большую часть - 80.2% в Центральном округе
</div>

### *Cтоимость чашки капучино при открытии*

In [ ]:
mos_cof_cup = moscow_coffee.pivot_table(index='district', values='middle_coffee_cup', aggfunc='median').reset_index()
mos_cof_cup.columns = ['district', 'median_coffee_cup_dis']
mos_cof_merge = mos_cof_cup.merge(moscow_coffee)
mos_cof_cup.sort_values(by='median_coffee_cup_dis', ascending=False)

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
mos_cofeee = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(mos_map)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster

Choropleth(
    geo_data=state_map,
    data=mos_cof_merge,
    columns=['district', 'median_coffee_cup_dis'],
    key_on='feature.name',
    fill_color='RdPu',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(mos_cofeee)

marker_cluster = MarkerCluster().add_to(mos_cofeee)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']}  {row['rating']} {row['middle_coffee_cup']} {row['is_24/7']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
mos_cof_merge.apply(create_clusters, axis=1)

# выводим карту
mos_cofeee

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">

Наблюдаем, что самые высокие цены за чашку капучино в ЦАО, ЗАО и ЮЗАО, а самые маленькие цены в ВАО.
</div>

## **Вывод**

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid green 2px; padding: 20px">
<br>
<b>1. Открыли файлы и изучили данные</b>
    
- было обнаружено множество пропусков
- были обнаружено много некорректных данных на первый взгляд

<b>2. Провели предобработку</b>
    
- привели названия колонок к нижнему регистру и переименовали как удобно
- заполнили пропуски заглушками где смогли
- ввели два новых столбца `is_24/7` и `street`
- избавились от неявных дубликатов, где смогли найти
- исправили аномальные значения и заполнили пропуски медианой по категориям
    
<b>3. Проанализировали данные</b>

- выяснили количество заведений по категориям
- выяснили минимальную и максимальную дату и отрезок проведения эксперимента
- выяснили наиболее часто встречающиеся количество мест в заведениях по категориям
- выяснили доли сетевых/несетевых заведений
- выяснили доли сетевых заведений по категориям
- выяснили ТОП-15 сетевых заведений
- выяснили какое количество заведений в категориях по административным округам
- сделали распределение средних рейтингов по категориям заведений
- построили фоновую картограмму заведений
- выяснили ТОП-15 улиц по количеству заведений
- выяснили какие улицы с одним объектом общепита
- выяснили средние чеки по административным округам
    
<b>4. Детализировали исследование для открытия кофейни</b>

- выяснили количество кофеен
- выяснили рейтинг кофеен по административным округам
- выяснили количество кофеен, которые работают 24/7
- выяснили количество сетевых и несетевых кофеен, которые работают 24/7 и рассчитали их долю
- выяснили какую стоимость определять для чашки капучино при открытие кофейни
   

    
<br>
<br>
<b>Целью исследования было:</b> найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места
    

<br>
<br>
<b>В ходе проведения исследования сделали выводы:</b>

1. Среди категорий заведений популярны такие категории: кафе, ресторан, кофейни. Рынок заведений полон этими категориями, следовательно там больше конкуренции.
2. Рестораны, бары/пабы и кофейни обычно требуют больше всего посадочных мест, значит площадь заведения должна быть большая.
3. Несетевых заведений в Москве больше, процентные пункты составляют `61.9%`
4. Булочные, пиццерии и кофейни занимают большую часть сетевых заведений
5. В Центральном административном округе больше всего количества заведений, то есть конкуренция там высокая. В Северо-Западном административном округе меньше всего заведений следовательно и конкуренции там будет меньше.
4. Рейтинг заведения по категориям выглядит примерно одинаковым по всем категориям, но булочные, столовые и заведения быстрого питания меньше всего получают самые высокие оценки.
5. Протяженность улицы зависит на количество на ней заведений, чем длиннее улица, тем больше заведений на ней.
6. В парковых, набережных и зонах рядом железной дорогой чаще бывает одно заведение.
7. Средний чек в заведениях чаще всего зависит от особенностей округа

<br>
<br>

<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid orange 2px; padding: 20px">
<b>Рекомендации при открытие кофейни:</b>

Кофейню лучше всего открывать в местах, где лучше всего проходимость. Проходимость в Центральном округе больше всего.
    
<b>Среди самых проходимых улиц(данные из исследования Геомаркетингового агенства One by One | 1by1):</b>
- ул. Маросейка
- пр. Мира
- ул. Бауманская
- ул. Тверская
    

Необязательно открывать кофейню именно на этих улицах, можно присмотреться к близлежайшим улицам, переулкам, проездам.
    
В Центральном административном округе средняя цена за чашку капучино 190р. Так как заведение будет новым, рекомендую снизить цену на 10-20 рублей, а в дальнейшем с течением времени уже обновить прайс.

Делать круглосуточную кофейню не стоит, так как эту нишу заняли сетевые кофейные у которых уже все процессы настроены.
Открываться по френчайзингу в теории можно, есть выгода за счёт настроенных процессов, рекламы, бренда, но вы лишаетесь индивидуальности и регламентов, которые могут быть неподходящими под вашу философию. При открытие открытие по френчайзингу - это заведение не будет вашим детищем, а вы там будете как инвестор с долей менеджмента.
</div>
<br>
<br>
<div style="border-radius: 15px; box-shadow: 4px 4px 4px; border: solid red 2px; padding: 20px">
<b>Ссылка на презентацию:</b>
<a href="https://disk.yandex.ru/i/TC4cQtO8cfFlmw">ТЫК</a>
</div>
</div>